In [ ]:
import numpy as np

def bit_func(x, y):
  return x | ~y & x

class GDNeuralNetwork:
    """
    """

    def __init__(self, weights=None):
        # вектор весов
        self.w0 = 0
        self.w1 = 0
        self.w2 = 0
        self.w3 = 0
        self.w4 = 0
        self.w5 = 0
        self.w6 = 0
        self.w7 = 0
        self.w8 = 0

        self.b1 = 1
        self.b2 = 1
        self.b3 = 1


    def sigmoid(self, x):
        # Функция активации sigmoid: f(x) = 1 / (1 + e^(-x))
        return 1 / (1 + np.exp(-x))

    def deriv_sigmoid(self, x):
        # Производная от sigmoid: f'(x) = f(x) * (1 - f(x))
        fx = self.sigmoid(x)
        return fx * (1 - fx)

    def mse_loss(self, y_true, y_pred):
        # y_true и y_pred являются массивами numpy одинаковой длины
        return ((y_true - y_pred) ** 2).mean()

    def get_weights(self):
        return self.w0, self.w1,self.w2,self.w3,self.w4,self.w5,self.w6,self.w7,self.w8

    def feedforward(self, x):
        # x является массивом numpy с двумя элементами
        h1 = self.sigmoid(self.w0 * x[0] + self.w2 * x[1] + self.w4 * self.b1)
        h2 = self.sigmoid(self.w1 * x[0] + self.w3 * x[1] + self.w5 * self.b2)
        o1 = self.sigmoid(self.w6 * h1 + self.w7 * h2 + self.w8 * self.b3)
        return o1

    def class_predict(self, x):
        return f"С вероятностью {round(self.feedforward(x) * 100, 3)}% это 1\n" + \
               f"С вероятностью {round((1 - self.feedforward(x)) * 100, 3)}% это 0"

    def train(self, data, all_y_trues, epochs):
        """
        """
        learn_rate = 0.3

        for epoch in range(epochs):
            for x, y_true in zip(data, all_y_trues):
                # --- Выполняю обратную связь (эти значения понадобятся в дальнейшем)
                sum_h1 = self.w0 * x[0] + self.w2 * x[1] + self.w4 * self.b1
                h1 = self.sigmoid(sum_h1)

                sum_h2 = self.w1 * x[0] + self.w3 * x[1] + self.w5 * self.b2
                h2 = self.sigmoid(sum_h2)

                sum_h3 = self.w2 * x[0] + self.w5 * x[1]
                h3 = self.sigmoid(sum_h3)

                sum_o1 = self.w6 * h1 + self.w7 * h2 + self.w8 * self.b3
                o1 = self.sigmoid(sum_o1)

                y_pred = o1

                # --- Подсчет частных производных
                # --- Наименование: d_L_d_w1 представляет "частично L / частично w1"
                d_L_d_ypred = -2 * (y_true - y_pred)

                # Нейрон o1
                d_ypred_d_w7 = h1 * self.deriv_sigmoid(sum_o1)
                d_ypred_d_w8 = h2 * self.deriv_sigmoid(sum_o1)
                d_ypred_d_w9 = self.b3 * self.deriv_sigmoid(sum_o1)

                d_ypred_d_h1 = self.w6 * self.deriv_sigmoid(sum_o1)
                d_ypred_d_h2 = self.w7 * self.deriv_sigmoid(sum_o1)

                # Нейрон h1
                d_h1_d_w1 = x[0] * self.deriv_sigmoid(sum_h1)
                d_h1_d_w3 = x[1] * self.deriv_sigmoid(sum_h1)
                d_h1_d_w5 = self.b1 * self.deriv_sigmoid(sum_h1)

                # Нейрон h2
                d_h2_d_w2 = x[0] * self.deriv_sigmoid(sum_h2)
                d_h2_d_w4 = x[1] * self.deriv_sigmoid(sum_h2)
                d_h2_d_w6 = self.b2 * self.deriv_sigmoid(sum_h2)


                # --- Обновляю вес и смещения
                # Нейрон h1
                self.w0 -= learn_rate * d_L_d_ypred * d_ypred_d_h1 * d_h1_d_w1
                self.w2 -= learn_rate * d_L_d_ypred * d_ypred_d_h1 * d_h1_d_w3
                self.w4 -= learn_rate * d_L_d_ypred * d_ypred_d_h1 * d_h1_d_w5

                # Нейрон h2
                self.w1 -= learn_rate * d_L_d_ypred * d_ypred_d_h2 * d_h2_d_w2
                self.w3 -= learn_rate * d_L_d_ypred * d_ypred_d_h2 * d_h2_d_w4
                self.w5 -= learn_rate * d_L_d_ypred * d_ypred_d_h2 * d_h2_d_w6

                # Нейрон o1
                self.w6 -= learn_rate * d_L_d_ypred * d_ypred_d_w7
                self.w7 -= learn_rate * d_L_d_ypred * d_ypred_d_w8
                self.w8 -= learn_rate * d_L_d_ypred * d_ypred_d_w9

            # --- Подсчитываю общую потерю в конце каждой фазы
            if epoch % 10 == 0:
                y_preds = np.apply_along_axis(self.feedforward, 1, data)
                loss = self.mse_loss(all_y_trues, y_preds)
                print("Epoch %d loss: %.3f" % (epoch, loss))


# Определение набора данных
data = np.array([
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1],
])

expected_y = np.array([
    0,
    0,
    1,
    1
])

# Тренирую нейронную сеть
network = GDNeuralNetwork()
network.train(data, expected_y, 3000)
# Веса
print("w")
print(network.get_weights())
# Предсказания сети
print(network.class_predict([0, 0]))

print(network.class_predict([1, 0]))


Epoch 0 loss: 0.250
Epoch 10 loss: 0.251
Epoch 20 loss: 0.251
Epoch 30 loss: 0.251
Epoch 40 loss: 0.251
Epoch 50 loss: 0.251
Epoch 60 loss: 0.250
Epoch 70 loss: 0.250
Epoch 80 loss: 0.249
Epoch 90 loss: 0.247
Epoch 100 loss: 0.244
Epoch 110 loss: 0.238
Epoch 120 loss: 0.228
Epoch 130 loss: 0.212
Epoch 140 loss: 0.189
Epoch 150 loss: 0.160
Epoch 160 loss: 0.130
Epoch 170 loss: 0.101
Epoch 180 loss: 0.077
Epoch 190 loss: 0.059
Epoch 200 loss: 0.047
Epoch 210 loss: 0.037
Epoch 220 loss: 0.030
Epoch 230 loss: 0.025
Epoch 240 loss: 0.022
Epoch 250 loss: 0.019
Epoch 260 loss: 0.016
Epoch 270 loss: 0.014
Epoch 280 loss: 0.013
Epoch 290 loss: 0.012
Epoch 300 loss: 0.010
Epoch 310 loss: 0.010
Epoch 320 loss: 0.009
Epoch 330 loss: 0.008
Epoch 340 loss: 0.008
Epoch 350 loss: 0.007
Epoch 360 loss: 0.007
Epoch 370 loss: 0.006
Epoch 380 loss: 0.006
Epoch 390 loss: 0.005
Epoch 400 loss: 0.005
Epoch 410 loss: 0.005
Epoch 420 loss: 0.005
Epoch 430 loss: 0.004
Epoch 440 loss: 0.004
Epoch 450 loss: 0.004